# Homework 1: Named Entity Recognition (NER) with Sequence Labeling Models
## CS4740/5740 Fall 2022

### Milestone Submission Due: **September 15, 2022 (11:59 PM)**

### Project Submission Due: **September 27, 2022 (11:59 PM)**

Should there be major updates to this document, we will announce them on [Ed Stem](https://edstem.org/us/courses/26793/discussion/).  Minor updates are documented on [this particular much-beloved thread](https://edstem.org/us/courses/26793/discussion/1739296).






**Names:** Terrence Edson Lim

**Netids:** ttl39,




**Editing your version of this notebook:** One partner should make a copy of this notebook and share it with your partner.  **However**, because of synchronization issues (even though Colab works with Google Drive), changes made in this notebook at the same time from different computers/browser windows may not save. We will go so far as to recommend that you close the tab with this notebook when you are not working on it so your partner doesn't face sync issues.



**Collaboration policy:** please be sure to check the collaboration policy on the [course website](https://courses.cs.cornell.edu/cs4740/2022fa/)!



> Assignment authors & testers: CS 4740/5740 professors and TAs from this and previous semesters, Chenxin Fang, Meghana Srivastava, Khonzoda Umarova, as well as Ruizhe Wang, Han Xia, and Heather Zhang.

# **Introduction**
---

In this project, you will tackle the **Named Entity Recognition** task: you'll implement models that identify named entities in text and tag them with the appropriate label. A primer on this task is provided further on.  We will treat this as a **sequence-tagging task**: for each token in the input text, assign one of the following 5 entity labels -- **ORG** (Organization), **PER** (Person), **LOC** (Location), **MISC** (Miscellaneous), and **O** (Not Named Entity) -- as well as a BIO-format prefix **B-** (token is the *beginning* of a named entity) or **I-** (token is *inside* a named entity). Overall, this yields 9 different labels: **B-ORG, I-ORG, B-PER, I-PER, B-LOC, I-LOC, B-MISC, I-MISC** and **O**.

For this project, you will implement two sequence labeling approaches:
- Model 1 : a Hidden Markov Model (HMM)
- Model 2 : a Maximum Entropy Markov Model (MEMM)/Logistic Regression classifier (also known as a MaxEnt classifier). Feature engineering is strongly suggested for this model!

A key component of both models is implementation of the Viterbi algorithm, which we will use to find the most likely tag sequence to assign to an input text.

## **Logistics**

---

- You are **strongly encouraged** to work in **groups of 2 students**. Students in the same group will get the same grade. Thus, you should make sure that everyone in your group contributes to the project.
- **Do not form teams of two on Kaggle** (You *will* form teams on a different platform; details TBA) because submitting separately gives you more tries, which is useful when experimenting with different models.
- [CORRECTED] You no longer need to have your netids(s) in your "Kaggle team" name. **We encourage you to change to/pick a fun Kaggle submission name. (Be reasonable, of course :) )**  You can change the team name under `Team` tab of the competition. Remember: since you and your partner are not forming your group on the Kaggle platform, this "team" would actually only include you.
- A part of your submission would involve uploading your notebook (details would be provided soon!). So, please enter all code and answer all the questions in this colab notebook.
  - Your code must have docstrings/comments documenting the meaning of parameters and important parameter-like variables.
- In this assignment you are asked to make two submissions:
  1. Intermediate **milestone submission due on 9/15/22 (11:59 PM)**. **Please see the HW1 Milestone assignment on CMS for instructions!** For this, please 1) have your teams formed on CMS and  2) submit predictions of your first model (HMM) on Kaggle. This means that you should aim to complete Part 1 and Part 2 of the assignment by this milestone. Points will be awarded for meeting the milestone deadline, but we will only grade for completion, not correctness.
  2. The **final homework submission due on 9/27/22 (11:59 PM)**. (details TBA)
- Please be sure to consult the [list](https://docs.google.com/document/d/1-QmpkZYJDCM4gQQ9sZW2EwD5ltYy1CFjlJTbhD0Be-A/edit?usp=sharing) of banned packages/libraries before you start implementing your models. Note that this list may get updated.
- [ADDED Wed Sep 14] You may not change any function headers/specifications that we have supplied.



## **Advice**

---

1. Please read through the entire notebook before you start coding. That might inform your code structure.
2. An assignment outline and grading breakdown (subject to minor adjustments) is found below; please consult it.
3. The project is somewhat open ended. We will ask you to implement the models, but in some cases precise data structures and so on can be chosen by you. However, to integrate with Kaggle, you will need to submit Kaggle predictions using the given evaluation code (more instructions later).

<a name="outline"></a>
## **Assignment outline and grading breakdown**
- [Part 1](#part1)
  - [Q1](#q1) [10 pts]
- [Part 2](#part2)
  - [Unknown Word Handling](#unknowns_handling) [15 pts]
  - [HMM Implementation](#hmm_implementation) [20 pts]
  - [Viterbi Implementation](#viterbi_implementation) [20 pts]
  - [Validation](#validation_data) [3 pts]
  - [Q2.1](#q2.1) [5 pts]
  - [Q2.2](#q2.2) [5 pts]
  - [Q2.3](#q2.3) [5 pts]
  - [Q2.4](#q2.4) [5 pts]
- [Part 3](#part3)
  - [Features](#features) [35 pts]
  - [MEMM Implementation](#memm_implementation) [25 pts]
  - [Q3.1](#q3.1) [5 pts]
  - [Q3.2](#q3.2) [5 pts]
  - [Q3.3](#q3.3) [5 pts]
  - [Q3.4](#q3.4) [5 pts]
- [Part 4](#part4)
  - [Q4.1](#q4.1) [7 pts]
  - [Q4.2](#q4.2) [7 pts]
  - [Q4.3](#q4.3) [7 pts]
- [Part 5](#part5)
  - [Q5](#q5)


Meeting the milestone deadline [10 pts];

Outperforming our baseline on Kaggle [15 pts];



## **Named Entity Recognition: Review**

---

NER refers to the information extraction technique of identifying and categorizing key information about entities within textual data. NER is important for:
  - Detecting entities in search engines and voice assistants for more relavent search results.
  - Automatically parsing resumes.
  - ...and much more!

In our dataset named entity tags are formatted in BIO/IOB format. With this format, entity tags get a prefix. Prefix "B-" is added to the first word/token of the entity name. All following tokens that are part of the same entity name would get prefix "I-".

Here is an example sentence: "ZIFA
said
Renate
Geotschel
of
Austria
won the women's World Cup  downhill race in Germany."
Entity "Renate Goetschl" gets "Renate" (B-PER) and "Goestchl" (I-PER). Similarly, for "World Cup" we'd have "World" (B-MISC) and "Cup" (I-MISC). If an entity only has one token, then its entity tag would still have prefix "B-". "O" is used to denote tokens that are not part of any named entity. Thus, from the example above, we'd have:

```"ZIFA" B-ORG```

 ```"said" O```

 ```"Renate" B-PER```

 ```"Goetschl" I-PER```

 ```"of" O```

 ```"Austria" B-LOC```

 ```"won" O```

 ```"the" O```

 ```"women's" O```

 ```"World" B-MISC```

 ```"Cup" I-MISC```

 ```"downhill" O```

 ```"race" O```

 ```"in" O```

 ```"Germany" B-LOC```


Although NER is predominantly handled by deep learning approaches, for now let's use HMMs and MEMMs.


To read more on NER, we refer to any of the following sources:
1. Medium post [1](https://umagunturi789.medium.com/everything-you-need-to-know-about-named-entity-recognition-2a136f38c08f) and [2](https://medium.com/mysuperai/what-is-named-entity-recognition-ner-and-how-can-i-use-it-2b68cf6f545d).
2. Try out [this](https://demo.allennlp.org/named-entity-recognition/named-entity-recognition) AlllenNLP demo! Please note that this demo uses a slightly different format of NER tags.

## **Evaluation: Entity Level Mean F1**

---

The standard evaluation measures to report for NER are recall, precision, and F1 score
(also called F-measure) evaluated at the **named entity level** (not at the token level). The code for this has been provided later under the validation section under Part 2. Please use this code when evaluating your models.


If P and T are the sets of predicted and true *named entity spans*, respectively, (e.g, the five named entity spans in the above example are "Zifa", "Renate Goetschl", "Austria", "World Cup", and "Germany") then

####<center>Precision = $\frac{|\text{P}\;\cap\;\text{T}|}{|\text{P}|}$ and Recall = $\frac{|\text{P}\;\cap\;\text{T}|}{|\text{T}|}$.</center><br/>


####<center>F1 = $\frac{2 * \text{Precision} * \text{Recall}}{\text{Precision} + \text{Recall}}$. </center><br/>

For each type of named entity, e.g. *LOC*ation, *MISC*ellaneous, *ORG*anization and *PER*son, we calculate the F1 score as shown above, and take the mean of all these F1 scores to get the **Entity Level Mean F1** score for the test set. If $N$ is the total number of labels (i.e., named entity types), then

####<center>Entity Level Mean F1 = $\frac{\sum_{i = 1}^{N} \text{F1}_{{label}_i}}{N}$. </center>

More details under the validation section in Part 2.



<a name="part1"></a>
# **Part 1: Dataset**
[[^^^]](#outline)

Our data is a modified version of the WikiNEuRal ([ Tedeschi et al.](https://aclanthology.org/2021.findings-emnlp.215.pdf)) dataset.

Load the dataset as follows:
  1. Obtain the data from Data tab of the [Kaggle competition](https://www.kaggle.com/t/200697e4726f448986930dd4e823e957).
  2. Unzip the data. Put it into your Google Drive, run the cells below to mount it to Colab:


In [1]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import json

# TODO: please change the line below with your drive organization
path = os.path.join(os.getcwd(), "drive", "MyDrive")

with open(os.path.join(path,'train.json'), 'r') as f:
     train = json.loads(f.read())

with open(os.path.join(path,'val.json'), 'r') as f:
     val = json.loads(f.read())

with open(os.path.join(path,'test.json'), 'r') as f:
     test = json.loads(f.read())

Here's a few things to note about the dataset above:
1. We have loaded 3 `.json` files: for training, validation, and testing.
2. The train and validation files contain the following 3 fields (each is a nested list):
  - **'text'** - actual input tokens
  - **'NER'** - the token-level entity tag
  - **'index'** - index of the token in the dataset
3. The test data only has **'text'**, and **'index'** fields. You will need to submit your prediction of the **'NER'** tag to Kaggle.


### **Q1: Initial Data Observations**

In the space below please add your code for dataset explorations for Q1.

In [3]:
from numpy.lib.shape_base import take_along_axis

# Printing the fist sentence with entitity-tokens.
out_string = ''
for i in range(len(train.get('text')[0])):
  tag = train.get('NER')[0][i]
  if tag != "O":
    out_string += '[' + tag + '] '
  out_string += train.get('text')[0][i]
  out_string += ' '
print(out_string + '\n')

# Printing some other sentence with entitity-tokens.
out_string = ''
for i in range(len(train.get('text')[4921])):
  tag = train.get('NER')[4921][i]
  if tag != "O":
    out_string += '[' + tag + '] '
  out_string += train.get('text')[4921][i]
  out_string += ' '
print(out_string + '\n')

print('train # sentences: ' + str(len(train.get('text'))) + '\n')
print('val # sentences: ' + str(len(val.get('text'))) + '\n')


# Printing types of entity-tokens and building dict entity_freq
out_str = ''
data_size = train.get('text')
entity_freq = {} # ***IMPORTANT***

for i in range(len(data_size)):
  sentence = data_size[i]
  for j in range(len(sentence)):
    tag = train.get('NER')[i][j]
    if tag not in entity_freq:
      out_str += tag + ', '
      entity_freq.update({tag : 1})
    else:
      entity_freq.update({tag : entity_freq.get(tag) + 1})
print('Types of entity-tokens: ' + out_str + '\n')

# Keeping track of number unique words in train.
out_str = ''
unique_words = set()
  # words are keys and values are the count of that word in train.json ***IMPORTANT***
word_counter_train = {} # ***IMPORTANT***
for i in range(len(data_size)):
  sentence = data_size[i]
  for j in range(len(sentence)):
    word = train.get('text')[i][j]
    token = train.get('NER')[i][j]
    if word not in unique_words:
      unique_words.add(word)
    if word not in word_counter_train:
      word_counter_train.update({word : 1})
    else:
      word_counter_train.update({word : word_counter_train.get(word) + 1})

# number of total words in train
tot_words_t = len(unique_words)
print('Total number of words in train.json: ' + str(tot_words_t) + '\n')

# number of total words in val
out_str = ''
data_size = val.get('text')
unique_words = set()
  # words are keys and values are the count of that word in val.json ***IMPORTANT***
word_counter_val = {} # ***IMPORTANT***

for i in range(len(data_size)):
  sentence = data_size[i]
  for j in range(len(sentence)):
    word = val.get('text')[i][j]
    if word not in unique_words:
      unique_words.add(word)
    if word not in word_counter_val:
      word_counter_val.update({word : 1})
    else:
      word_counter_val.update({word : word_counter_val.get(word) + 1})

# Printing the count of each entity-token
print('Count of entity tokens (train):\n')
  # total number of tokens in train.json
sum_tokens = 0 # ***IMPORTANT***
for token in entity_freq:
  count = entity_freq.get(token)
  print('  ' + token + ': ' + str(count))
  sum_tokens += count

print('\n')
# Printing the frequency of each entity-token
print('Frequency of entity tokens (train):\n')
  # total number of tokens in train.json
for token in entity_freq:
  count = entity_freq.get(token)
  token_freq = count / sum_tokens
  entity_freq.update({token : token_freq})
  print('  ' + token + ': ' + str(token_freq))

# word_counter_train

Because of its high rate of economic marginalization , more people migrate from [B-LOC] Chiapas than migrate to it . 

Within the first six months of [B-LOC] Abbasid rule , revolts began erupting in the city , albeit too isolated and unfocused to present a viable threat . 

train # sentences: 7000

val # sentences: 400

Types of entity-tokens: O, B-LOC, B-PER, I-PER, B-ORG, B-MISC, I-LOC, I-MISC, I-ORG, 

Total number of words in train.json: 24823

Count of entity tokens (train):

  O: 147113
  B-LOC: 4060
  B-PER: 3016
  I-PER: 2217
  B-ORG: 1725
  B-MISC: 2815
  I-LOC: 1451
  I-MISC: 2566
  I-ORG: 1431


Frequency of entity tokens (train):

  O: 0.8841244275634939
  B-LOC: 0.02439991826628364
  B-PER: 0.018125653569239276
  I-PER: 0.013323797733091338
  B-ORG: 0.0103669603471279
  B-MISC: 0.016917677320095678
  I-LOC: 0.00872026635575802
  I-MISC: 0.015421229130858085
  I-ORG: 0.008600069714052189


<a name="q1"></a>
[[^^^]](#outline)

What are your initial observations after you explore the dataset?  Provide some quantitative data exploration. Assess dataset size, document lengths and the token-level NER class distribution, and the entity-level NER class distribution (skipping the 'O' label for the latter). Give some examples of sentences with their named entities bracketed, e.g. [[B-LOC Romania] state budget soars in June .] and [[B-ORG Zifa] said [B-PER Renate] [I-PER Goetschl] of [B-LOC Austria]...].


#### **A1:**

There are 8602 commas in train.  There are 25 occurences of 'had' in val.

train # sentences: 7000

val # sentences: 400

Types of entity-tokens: B-LOC, B-PER, I-PER, B-ORG, B-MISC, I-LOC, I-MISC, I-ORG,

Total number of words in train.json: 24823

Count of entity tokens:

  O: 147113
  B-LOC: 4060
  B-PER: 3016
  I-PER: 2217
  B-ORG: 1725
  B-MISC: 2815
  I-LOC: 1451
  I-MISC: 2566
  I-ORG: 1431

<a name="part2"></a>
# **Part 2: Hidden Markov Model**
[[^^^]](#outline)
---
In this part of the assignment, you will:
1. Implement code for counting and smoothing of labels and words, as well as unkown word handing, as necessary to support the Viterbi algorithm.
2. Build a Hidden Markov Model in accordance with the provided function headers. **You may NOT change the function specifications.** Please ensure that your code is clear, concise, and, most important of all, modular. This means you should break your implementation down into smaller functions or write it within a class. Please compute all probabilities in natural log when building the HMM.
3. Implement the **Viterbi algorithm**, that can be used to infer token-level labels (identifying the appropriate named entity) for an input document. This process is commonly referred to as **decoding**. Bigram-based Viterbi is $ \mathcal{O}(sm^2)$ where *s* is the length of the sentence and *m* is the number of tags. Your implementation should have similar efficiency. The code for this can be used later on for the MEMM too.

### References
You may find chapters [3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) and [8](https://web.stanford.edu/~jurafsky/slp3/8.pdf) of Jurafsky and Martin book useful. In particular, section 3.4.1 covers ways to handle unknown words, and section 3.5 goes over smoothing.

<a name="unknowns_handling"></a>
## **Unknown Word Handling**
[[^^^]](#outline)
---

In [4]:
import math

#Define vocabulary list as all the unique tokens that appear in the training dataset
#Add a token <unk> to the vocabulary list to deal with unknown words
#Add <unk> to the emissions and start matrices setting probability to 0
#Smooth the matrices
#Implement add-k smoothing when building HMM
#In viterbi algorithm, when an unknown token is encountered, change it to <unk>
#so we still see that token in the emissions matrix and get an output probability


#Define vocabulary list as all the unique tokens that appear in list of sentences
#Use train.get('text') as attribute to make all words in the training dataset the vocab
#Add <unk> as a vocab word
#We can do these steps using the method below

#Finds every unique element in the list of lists and add an <unk> element.
def find_unique(list_of_lists):
  uniquelist = []
  for sentence in list_of_lists:
    for token in sentence:
      if token not in uniquelist:
        uniquelist.append(token)
  uniquelist.append('<unk>')
  return uniquelist

def find_unique_no_unknown(list_of_lists):
  uniquelist = []
  for sentence in list_of_lists:
    for token in sentence:
      if token not in uniquelist:
        uniquelist.append(token)
  return uniquelist







<a name="hmm_implementation"></a>
## **HMM Implementation**
[[^^^]](#outline)
---

In the skeleton code below, we have broken down the HMM into its three components: the transition matrix, the emission matrix, and the start state probabilities. We suggest you implement them separately and then use them to build the HMM.


In [5]:
from ast import AsyncFunctionDef

#Returns the transition probabilities
def build_transition_matrix(labels, k=0):
  """
    Returns a dictionary that has tuples of every label bigram as keys, and
    the associated value being the respective transition probabilities (in
    natural log).

    Eg. {("O", "B-ORG"): -9.98690147425591,
         ("B-LOC", "I-LOC"): -3.69537214,
         ...,
         ...,
        }

    :parameter labels: A list where each element represents a sentence,
    and each sentence is a list of NER labels for each of its tokens. (Eg.
    [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], [...], ...])
    :type labels: List[List[String]]
    :parameter k: an optional parameter for smoothing
    :type k: int
    """
  #bigram_lexicon: Keys are unique tuples of every label bigram,
  #bigram_lexicon: Values are the total occurences of the keys
  bigram_lexicon = {}
  #labels_count: Keys are the unique start labels
  #labels_count: Values are the total occurences of the keys (unique labels)
  labels_count = {}

  #loop through all the sentences in labels
  for sentence in labels:
    #loop through every pair of words in each sentence
    for tag_index in range(1, len(sentence)):
      prev_tag_index = tag_index - 1
      #current pair of words (bigram)
      bigram = (sentence[prev_tag_index], sentence[tag_index])
      #if the bigram isn't in our bigram_lexicon, we must add
      #with a total occurence of 1.
      if bigram not in bigram_lexicon:
        bigram_lexicon.update({bigram : 1})
      #if the bigram is already in bigram_lexicon, we must add 1 to its count.
      else:
        update_count = bigram_lexicon.get(bigram) + 1
        bigram_lexicon.update({bigram : update_count})
      #update the labels_count
      prev_tag = sentence[prev_tag_index]
      if prev_tag not in labels_count:
        labels_count.update({prev_tag : 1})
      else:
        new_count = labels_count.get(prev_tag) + 1
        labels_count.update({prev_tag : new_count})

  #Smooth the data for unknown context. So if n is the amount of labels present,
  #where <unk> is included as a possible label,
  #we should have n^2 total bigrams accounted for.

  #Determine all the types of labels
  uniquelabels = find_unique_no_unknown(labels)
  #make sure that our bigram_lexicon incorporates every comination of label to label
  for label1 in uniquelabels:
    for label2 in uniquelabels:
      newbigram = (label1, label2)
      if newbigram not in bigram_lexicon:
        bigram_lexicon.update({newbigram : 0})
  #Apply add-k smoothing to the now complete bigram_lexicon
  for (label1, label2) in bigram_lexicon:
    bigram = (label1, label2)
    new_count = bigram_lexicon.get(bigram) + k
    bigram_lexicon.update({bigram : new_count})
    new_labels_count = labels_count.get(label1) + k
    labels_count.update({label1 : new_labels_count})

  #bigram_lexicon_logs: Keys are unique tuples of every label bigram,
  #bigram_lexicon_logs: Values are the log probabilities of the unique bigram
  bigram_lexicon_logs = {}

  #Get the log probabilities for every bigram
  for (label1, label2) in bigram_lexicon:
    bigram = (label1, label2)
    log_prob = math.log(bigram_lexicon.get(bigram) / labels_count.get(label1))
    bigram_lexicon_logs.update({bigram : log_prob})

  return bigram_lexicon_logs

#labels = [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']]
#build_transition_matrix(labels, .000000000000000001)

In [6]:

# Returns the emission probabilities.
def build_emission_matrix(tokens, labels, k=0):
  """
    Returns a dictionary that has label-token tuples as keys, and emission
    probabilities (in natural log) for each respective label-token pair as values.

    Eg. {("O", "Because"): -10.133904545421267,
         ("I-PER", "Markov"): -7.428569227340841,
         ...,
         ...,
        }

    :parameter tokens: A list where each element represents a sentence,
    and each sentence is a list of its tokens. (Eg. [['The', 'most',
    'significant', 'damage', 'was', 'on', 'Tortola', '.'], [...], ...])
    :type tokens: List[List[String]]
    :parameter labels: A list where each element represents a sentence,
    and each sentence is a list of NER labels for each of its tokens. (Eg.
    [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], [...], ...])
    :type labels: List[List[String]]
    :parameter k: an optional parameter for smoothing
    :type k: int
    """
  #label_token_lexicon: Keys are unique label-token tuples,
  #label_token_lexicon: Values are the total occurences of the keys
  label_token_lexicon = {}
  #labels_count: Keys are the unique labels
  #labels_count: Values are the total occurences of the keys (unique labels)
  labels_count = {}

  #loop through all the given sentences
  for sentence_i in range(len(labels)):
    sentence_tokens = tokens[sentence_i]
    sentence_labels = labels[sentence_i]
    #for each sentence, we loop through every token/tag
    for tag_i in range(len(sentence_tokens)):
      label = sentence_labels[tag_i]
      token = sentence_tokens[tag_i]
      label_token = (label, token)

      #update labels_count
      if label not in labels_count:
        labels_count.update({label : 1})
      else:
        new_count = labels_count.get(label) + 1
        labels_count.update({label : new_count})

      #update label_token_lexicon
      if label_token not in label_token_lexicon:
        label_token_lexicon.update({label_token : 1})
      else:
        new_count = label_token_lexicon.get(label_token) + 1
        label_token_lexicon.update({label_token : new_count})

  #Smooth the data for unknown context. So if X is the amount of labels present,
  #where <unk> is included as a possible label, and Y is the amount of tokens
  #present including the <unk> token, we should have X*Y total label_token
  #pairs accounted for.

  #Determine all the types of labels
  uniquelabels = find_unique_no_unknown(labels)
  #Determine all unique types of tokens
  uniquetokens = find_unique(tokens)
  #make sure that our bigram_lexicon incorporates every comination of label to token
  for label in uniquelabels:
    #update labels_count while we're at it
    if label not in labels_count:
      labels_count.update({label : 0})
    for token in uniquetokens:
      new_label_token = (label, token)
      if new_label_token not in label_token_lexicon:
        label_token_lexicon.update({new_label_token : 0})
  #Apply add-k smoothing to the now complete bigram_lexicon
  for (label, token) in label_token_lexicon:
    new_count = label_token_lexicon.get((label, token)) + k
    label_token_lexicon.update({(label, token) : new_count})
    new_labels_count = labels_count.get(label) + k
    labels_count.update({label : new_labels_count})

  #label_token_lexicon_logs: Keys are unique tuples of every label bigram,
  #label_token_lexicon_logs: Values are the log probabilities of the unique label token tuple
  label_token_lexicon_logs = {}

  #Get the log probabilities for every label token pair
  for (label, token) in label_token_lexicon:
    label_token = (label, token)
    fraction = label_token_lexicon.get(label_token) / labels_count.get(label)
    log_prob = math.log(fraction)
    label_token_lexicon_logs.update({label_token : log_prob})

  return label_token_lexicon_logs

#tokens = [['A', 'cat', 'eats', 'several', 'mice', 'in', 'Jamaica', '.'], ['Annie', 'and', 'family', 'are', 'returning', 'to', 'Germany', '.']]
#labels = [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']]
#build_emission_matrix(tokens, labels, k=.0000000001)

In [7]:
# Returns the starting state probabilities.
def get_start_state_probs(labels, k=0):
  """
    Returns a dictionary that has labels for keys, and the respective state
    probabilities (in natural log) for values.

    Eg. {"O": -10.133904545421267,
         "I-PER": -7.428569227340841,
         ...,
         ...,
        }

    :parameter labels: A list where each element represents a sentence,
    and each sentence is a list of NER labels for each of its tokens. (Eg.
    [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], [...], ...])
    :type labels: List[List[String]]
    :parameter k: an optional parameter for smoothing
    :type k: int
    """
  #lexicon: Keys are unique labels,
  #lexicon: Values are the total occurences of the keys (labels) at the start of
  #each sentence
  lexicon = {}
  #total number of labels at the start of a sentence
  total_start_labels = 0

  #loop through each sentence
  for sentence_i in range(len(labels)):
    #get the first word of each sentence and add its count to lexicon
    sentence = labels[sentence_i]
    start_label = sentence[0]
    if start_label not in lexicon:
      lexicon.update({start_label : 1})
    else:
      update_count = lexicon.get(start_label) + 1
      lexicon.update({start_label : update_count})
    total_start_labels += 1

  #Smooth the data for unknown context. So if n is the amount of unique labels present,
  #not necessarily at the start of a sentence and where <unk> is included as a
  #possible label, we should have n total unique start labels accounted for.

  #Determine all the types of labels
  uniquelabels = find_unique_no_unknown(labels)
  #make sure that our lexicon incorporates every label
  for label in uniquelabels:
    if label not in lexicon:
      lexicon.update({label : 0})
  #Apply add-k smoothing to the now complete lexicon
  for label in lexicon:
    new_count = lexicon.get(label) + k
    lexicon.update({label : new_count})
    total_start_labels += k

  #lexicon_logs: Keys are unique labels,
  #lexicon_logs: Values are the log probabilities of the unique label
  lexicon_logs = {}

  for label in lexicon:
    log_prob = math.log(lexicon.get(label) / total_start_labels)
    lexicon_logs.update({label : log_prob})

  return lexicon_logs

#labels = [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['I-PER', 'O']]
#get_start_state_probs(labels, k=.0000000000000000001)

In [8]:
# Takes in the tokens & labels and returns a representation of the HMM.
# Call the three functions above in this function to build your HMM.
def build_hmm(tokens, labels):
  """
    Returns a dictionary with keys: 'transitions', 'emissions', and
    'start_states', where the values of the three keys are the asssociated
    dictionary containing the transitions matrix, emissions matrix, and
    start_state matrix dictionaries for the tokens and labels, respectively.

    :parameter tokens: A list where each element represents a sentence,
    and each sentence is a list of its tokens. (Eg. [['The', 'most',
    'significant', 'damage', 'was', 'on', 'Tortola', '.'], [...], ...])
    :type tokens: List[List[String]]
    :parameter labels: A list where each element represents a sentence,
    and each sentence is a list of NER labels for each of its tokens. (Eg.
    [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], [...], ...])
    :type labels: List[List[String]]
    """
  hmm = {}
  #implement add-k smoothing
  k = .8
  transitions = build_transition_matrix(labels, k)
  emissions = build_emission_matrix(tokens, labels, k)
  start_states = get_start_state_probs(labels, k)
  hmm.update({'transitions' : transitions})
  hmm.update({'emissions' : emissions})
  hmm.update({'start_states' : start_states})

  return hmm

#tokens = [['A', 'cat', 'eats', 'several', 'mice', 'in', 'Jamaica', '.'], ['Annie', 'and', 'family', 'are', 'returning', 'to', 'Jamaica', '.']]
#labels = [['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['B-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O']]
#build_hmm(tokens, labels)

<a name="viterbi_implementation"></a>
## **Viterbi Implementation**
[[^^^]](#outline)
---

At the end of your implementation, we expect a function or class that maps a sequence of tokens (observation) to a sequence of labels via the Viterbi algorithm.

In [9]:
# Takes in the HMM built above and an observation (i.e. a list of tokens),
# and returns a list with predicted named entity mappings for the tokens.
# The returned list should be the same length as the input obeservation.
def viterbi(hmm, observation):

  #Get all components of the HMM
  transitions = hmm.get('transitions')
  #INITIAL DEBUGGING: print(transitions)
  emissions = hmm.get('emissions')
  start_states = hmm.get('start_states')
  #INITIAL DEBUGGING: print(start_states)

  #Get a list of all potential labels (Named Entity Mappings)
  labels_list = []
  for (label1, label2) in transitions:
    if label1 not in labels_list:
      labels_list.append(label1)
    if label2 not in labels_list:
      labels_list.append(label2)

  #INITIAL DEBUGGING: print(labels_list)

  #Initialization
  #label_tracker: tracks the previous label that got to this token.
  #Saves the previous label as a number representing that label's index in labels_list
  #list of lists, contains a list for ever token (the size of the inner list
  #is the total number of labels and contains the previous label that would get
  #current label for current token.)
  label_tracker = []
  #probabilities_tracker: tracks the probability to get to current token for every label
  #list of lists, contains a list for every token (size of inner list is the total
  #number of labels and contains the probability to get the current label for
  #the current token).
  probabilities_tracker = []

  #Initialize the start token
  #Initialize label_tracker for start token
  initial_label_tracker = [None] * len(labels_list)
  label_tracker.append(initial_label_tracker)
  #initialize probabilities_tracker for start token
  first_token = observation[0]
  first_token_probabilities = [] #probability first token has each label
  for label in labels_list:
    current_label_probability = 0
    #If the current word is not in the emissions, use '<unk>' to account for
    #unknown word. Else, we do the sum of the log start state probability and
    #the log emissions probability.
    if emissions.get((label, first_token)) is None:
      current_label_probability = start_states.get(label) + emissions.get((label, '<unk>'))
    else:
      current_label_probability = start_states.get(label) + emissions.get((label, first_token))
    first_token_probabilities.append(current_label_probability)
  probabilities_tracker.append(first_token_probabilities)

  #loop through every token
  for i in range(1, len(observation)):
    current_token = observation[i]

    #Saved max probability of each current_label_probability.
    #So we have the max probability to get to current label for the current token
    current_token_probability = []
    #Saves the previous label to get to the current label for the current token
    current_token_previous_label = []

    #Recursion process to fill in the probabilities_tracker and label_tracker
    #loop through all the labels to update probabilities current token has
    #the current label
    for current_label in range(len(labels_list)):
      #current_label_probability is the probability to have the current label
      #for the current token given all previous tokens as potential paths
      current_label_probability = []
      #loop through all the previous labels to see the paths to get to
      #this current label
      for previous_label in range(len(labels_list)):
        previous_label_to_current_label_probability = 0
        #probability = probability to get to previous label + transition + emissions
        if emissions.get((labels_list[current_label], current_token)) is None:
          previous_label_to_current_label_probability = \
          probabilities_tracker[i-1][previous_label] + \
          transitions.get((labels_list[previous_label], labels_list[current_label])) \
          + emissions.get((labels_list[current_label], '<unk>'))
        else:
          previous_label_to_current_label_probability = \
          probabilities_tracker[i-1][previous_label] + \
          transitions.get((labels_list[previous_label], labels_list[current_label])) \
          + emissions.get((labels_list[current_label], current_token))
        current_label_probability.append(previous_label_to_current_label_probability)
      #Get the max_probability_path to get to current label for current token
      max_probability_path = get_max_of_list(current_label_probability)
      #Save this max_probability_path to the current_token_probability and
      #the current_token_previous_label
      current_token_probability.append(max_probability_path[0])
      current_token_previous_label.append(max_probability_path[1])

    #Save the current_token_probability and current_token_previous_paths into
    #our overall probabilities_tracker and label_tracker
    #INITIAL DEBUGGING: print(current_token_probability)
    #INITIAL DEBUGGING: print(current_token_previous_label)
    probabilities_tracker.append(current_token_probability)
    label_tracker.append(current_token_previous_label)

  #Termination process to get the path
  last_token_index = len(observation)-1
  #Gets the highest probability path to the last token
  max_probability_last_token = get_max_of_list(probabilities_tracker[last_token_index])
  #Index of Label for the last token with the highest probability path
  last_label_index = max_probability_last_token[1]
  last_label = labels_list[last_label_index]

  #Get all previous labels given the last label.
  #Loop through all tokens starting with the last token and then we prepend
  #to list using python insert at index 0.
  current_label_index = last_label_index
  optimal_path = [last_label]
  for i in range(len(observation)-1):
    current_token = len(observation) - i  - 1
    previous_label_index = label_tracker[current_token][current_label_index]
    optimal_path.insert(0, labels_list[previous_label_index])
    current_label_index = previous_label_index

  return optimal_path



#returns a list where the first element is the value of the highest value in
#the list and the second value is the index of the highest value.
def get_max_of_list(list):
  current_index = 0
  current_max = list[0]
  for i in range(len(list)):
    if list[i] > current_max:
      current_max = list[i]
      current_index = i
  to_return = [current_max, current_index]
  return to_return





In [10]:
# Here's a sample observation that you can use to test your code
# Ending period added Wed Sep 14, 9am
obs_1 = ['Cornell',
 'University',
 'is',
 'located',
 'in',
 'Ithaca',
 'and',
 'was',
 'founded',
 'by',
 'Ezra',
 'Cornell',
 '.']

# Uncomment and fill out the following line to test your implementation:
hmm = build_hmm(train.get('text'), train.get('NER'))
viterbi(hmm, obs_1)

['B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O']

## **Validation Step**
<a name="validation_data"></a>
[[^^^]](#outline)
---

In this part of the project, we expect you to train your HMM model (i.e., get transition and emission probabilities) on the labeled training data and evaluate it on the validation data. Report **Entity Level Mean F1**, which was explained earlier. Please use the code we have provided below to compute this metric.

Please also take a look into your misclassified cases, as we will be performing error analysis in the *Evaluation* section. We expect smoothing, unknown word handling and correct emission (i.e., lexical generation) probabilities.

Consider the example below. After getting a sequence of NER labels for the sequence of tokens from your Viterbi algorithm implementation, you need to convert the sequence of tokens, associated token indices and NER labels into a format which can be used to calculate **Entity Level Mean F1**. We do this by finding the starting and ending indices of the spans representing each entity (as given in the corpus) and adding it to a list that is associated with the label with which the spans are labelled. To score your validation data on Google Colab or your local device, you can get a dictionary from the function **format_output_labels** on both the predicted and true label sequences, and use the two dictionaries as input to the **mean_f1** function.

NOTE: We do **not** include the spans of the tokens labelled as "O" in the formatted dictionary output.

In [11]:
def format_output_labels(token_labels, token_indices):
    """
    Returns a dictionary that has the labels (LOC, ORG, MISC or PER) as the keys,
    with the associated value being the list of entities predicted to be of that key label.
    Each entity is specified by its starting and ending position indicated in [token_indices].

    Eg. if [token_labels] = ["B-ORG", "I-ORG", "O", "O", "B-ORG"]
           [token_indices] = [15, 16, 17, 18, 19]
        then dictionary returned is
        {'LOC': [], 'MISC': [], 'ORG': [(15, 16), (19, 19)], 'PER': []}

    :parameter token_labels: A list of token labels (eg. B-PER, I-PER, B-LOC, I-LOC, B-ORG, I-ORG, B-MISC, OR I-MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset)
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = {"LOC":[], "MISC":[], "ORG":[], "PER":[]}
    prev_label = 'O'
    start = token_indices[0]
    for idx, label in enumerate(token_labels):
      curr_label = label.split('-')[-1]
      if label.startswith('B-') or curr_label != prev_label:
        if prev_label != 'O':
          label_dict[prev_label].append((start, token_indices[idx-1]))
        if curr_label != 'O':
          start = token_indices[idx]
        else:
          start = None

      prev_label = curr_label

    if start is not None and prev_label != 'O':
      label_dict[prev_label].append((start, token_indices[idx]))
    return label_dict

In [12]:
# Code for mean F1

import numpy as np

def mean_f1(y_pred_dict, y_true_dict):
    """
    Calculates the entity-level mean F1 score given the actual/true and
    predicted span labels.
    :parameter y_pred_dict: A dictionary containing predicted labels as keys and the
                            list of associated span labels as the corresponding
                            values.
    :type y_pred_dict: Dict<key [String] : value List[Tuple]>
    :parameter y_true_dict: A dictionary containing true labels as keys and the
                            list of associated span labels as the corresponding
                            values.
    :type y_true_dict: Dict<key [String] : value List[Tuple]>

    Implementation modified from original by author @shonenkov at
    https://www.kaggle.com/shonenkov/competition-metrics.
    """
    F1_lst = []
    for key in y_true_dict:
        TP, FN, FP = 0, 0, 0
        num_correct, num_true = 0, 0
        preds = y_pred_dict[key]
        trues = y_true_dict[key]
        for true in trues:
            num_true += 1
            if true in preds:
                num_correct += 1
            else:
                continue
        num_pred = len(preds)
        if num_true != 0:
            if num_pred != 0 and num_correct != 0:
                R = num_correct / num_true
                P = num_correct / num_pred
                F1 = 2*P*R / (P + R)
            else:
                F1 = 0      # either no predictions or no correct predictions
        else:
            continue
        F1_lst.append(F1)
    return np.mean(F1_lst)

In [13]:
# Usage using above example

pred_token_labels = ["B-ORG", "O", "B-PER", "I-PER", "O", "B-LOC", "O", "O", "O", "O", "B-MISC", "O", "O", "O", "O", "B-LOC"]
true_token_labels = ["B-ORG", "O", "B-PER", "I-PER", "O", "B-LOC", "O", "O", "O", "O", "B-MISC", "I-MISC", "O", "O", "O", "B-LOC"]
token_indices = [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

y_pred_dict = format_output_labels(pred_token_labels, token_indices)
print("y_pred_dict is : " + str(y_pred_dict))
y_true_dict = format_output_labels(true_token_labels, token_indices)
print("y_true_dict is : " + str(y_true_dict))

print("Entity Level Mean F1 score is : " + str(mean_f1(y_pred_dict, y_true_dict)))

y_pred_dict is : {'LOC': [(18, 18), (28, 28)], 'MISC': [(23, 23)], 'ORG': [(13, 13)], 'PER': [(15, 16)]}
y_true_dict is : {'LOC': [(18, 18), (28, 28)], 'MISC': [(23, 24)], 'ORG': [(13, 13)], 'PER': [(15, 16)]}
Entity Level Mean F1 score is : 0.75


In [14]:
pred_token_labels = []
true_token_labels = []
token_indices = []
hmm = build_hmm(train.get('text'), train.get('NER'))

for i in range(len(val.get('text'))):
  pred_token_labels += viterbi(hmm, val.get('text')[i])
  true_token_labels += val.get('NER')[i]
  token_indices += val.get('index')[i]

y_pred_dict = format_output_labels(pred_token_labels, token_indices)
#print("y_pred_dict is : " + str(y_pred_dict))
y_true_dict = format_output_labels(true_token_labels, token_indices)
#print("y_true_dict is : " + str(y_true_dict))

print("Entity Level Mean F1 score is : " + str(mean_f1(y_pred_dict, y_true_dict)))

Entity Level Mean F1 score is : 0.45509604788080804


<a name="q2.1"></a>
[[^^^]](#outline)
## **Q2.1: Explain your HMM Implementations**

Explain how you implemented the HMM including the Viterbi algorithm. Make clear which parts were implemented from scratch vs. obtained via an existing package (review the Logistics section for information on packages that are not allowed). Explain and motivate any design choices providing the intuition behind them.


#### **A2.1:**

Note comments on my HMM and Viterbi Algorithm. I created a label_tracker that tracks the previous label that got to this token. It saves the previous label as a number representing that label's index in labels_list (a list of every unique label). This is also a list of lists that cointains a list for every token (where the size of the inner list is the total number of labels and contains the previous label that would get the current label for the current token).

Then I had another variable probabilities_tracker: tracks the probability to get to current token for every label. It is a list of lists that contains a list for every token (size of inner list is the total number of labels and contains the probability to get the current label for the current token).

I then initialized by adding the first token to the probabilities tracker and previous label (all None since there's no previous token from first token)

Then, I looped through every word in the list to do the tagging. In each word, I looped through every possible tag for the current word. Then I looped through every possible previous tag that could get to the current tag for the current word. I calculated the probability using the emissions and transition matrix and saved the probability to probabilities_tracker.

Finally, once I have all the probabilities, I found the maximum probability path and backtracked using the label_tracker to get from the last label to the first label.

<a name="q2.2"></a>
[[^^^]](#outline)
## **Q2.2: Results Analysis**

Explain here how you evaluated the models. Summarize the performance of your system and any variations that you experimented with on the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis.


#### **A2.2:**

... add your answers here

I evaluated my models and performance of my system by using the validation data and the calculated Entity Level Mean F1 Score. The implementation and viterbi was very straight forward for the most part, and the model stayed the same. However, the only variable I changed was when I applied add-k smoothing. I experimented by changing different values of k to test which would have a lower Entity Level Mean F1 Score.

DATA:
If k=0.25, Entity Level Mean F1 Score on Validation Data = 0.526
If k= 0.25,


<a name="q2.3"></a>
[[^^^]](#outline)
## **Q2.3: Error Analysis**
When did the system work well? When did it fail?  Any ideas as to why? How might you improve the system?


#### **A2.3:**

... add your answers here

The system worked well when the add-k smoothing had a low k value. The system worked less efficiently when there was a higher value for add-k smoothing. I believe the reason is that there are a lot of unknown words, and having a high k value can skew the data from common words to obscure words.

The system failed when we did add-0 smoothing. The reason is that we added all the tags and different combinations in our emissions matrix that could show up with a probability of 0 before smoothing, so these would translate to an error when calculating the log probabilities. Thus, k must be greater than 0.

I might improve the system by implementing a different version of smoothing that could be more efficent. I might also be able to improve the system by using the naive-bayes classifer as a possible probability to get a more efficient result.

<a name="q2.4"></a>
[[^^^]](#outline)
## **Q2.4: What is the effect of unknown word handling and smoothing?**

#### **A2.4:**

... add your answers here

The effect of unknown word handling and smoothing is that when the data encounters and known situation such as a new word, rather than immediately ruling that path out, we can assign a small probability to account for the fact that our training dataset does not account for the entire sample space. Also, smoothing helps us account for situations and combinations of words in a context that we have not seen. By doing so, we assign a small probability for these rare but possible events in our test data.

Uknown word handling and smoothing can affect our data b/c if we have a very high value for add-k smoothing ( a high k), our data can be skewed in the wrong way resulting in a worse result. However, we need to find the balance when validating our data to get the best results.

<a name="part3"></a>
# **Part 3: Maximum Entropy Markov Model**
[[^^^]](#outline)
---

In this section, you will implement a Maximum Entropy Markov Model (**MEMM**) to perform the same NER task. Your model should consist of a MaxEnt classifier with Viterbi decoding.

1. We have already performed tokenizations for documents. You can either use a MaxEnt classifier from an existing package or write the MaxEnt code yourself. **Important note:  MaxEnt classifiers are statistically equivalent to multi-class logistic regression, so you can use packages for multi-class LR instead of MaxEnt.**

2. Use the classifier to learn a probability $P(t_i|features)$. You may replace either the lexical generation probability – $P(w_i|t_i)$ – or the transition probability – $P(t_i|t_{i−1})$ – in the HMM with it, or you may replace the entire *lexical generation probability * transition probability*  calculation – $P (w_i|t_i) ∗ P (t_i|t_{i−1)} – $ in the HMM with it.

3. To train such a classifier, you need to pick some feature set. The content of the feature set is up to you, but try different ones, and evaluate your choices on the validation set. Pick the feature set that performs overall the best according to the F1 measure. If you draw inspiration for your features from external sources, please link them in the code.
  * While there are many directions to take when looking for features, you may start by exploring parts of speech that appear in sentences. There are several libraries (ex. [nltk](https://www.nltk.org/book/ch05.html)) that process sentences and identify parts of speech. If you end up using a library to extract parts of speech tags or other features, please indicate this in your asnwer to Q3.1.

4. Use your own implementation of the **Viterbi algorithm**, which you can modify from the one you developed for the HMM model. You will need the probabilities that you obtain from the MaxEnt classifier.

5. Remember to use same training and validation split when evaluating the MEMM to have a **fair comparison** with your **HMM model**.


Please also take a look into your misclassified cases, as we will be performing error analysis in Part 4.





---
Here's a summary of the workflow for Part 3:

![alt text](https://drive.google.com/uc?export=view&id=14VfjW3yDyXLojWM_u0LeJYdDOSLkElBn)




Note that we have not provided any skeleton code for how you should do feature engineering since this is meant to be an open ended task and we want you to experiment with the dataset. However, please remember to make sure that you code is concise, clean, and readable! Ultimately, we expect a function or class  mapping a sequence of tokens to a sequence of labels. At the end of this section you should have done the following:
1. Extract Features
2. Build & Train MaxEnt
3. Call Viterbi when evaluating

### References
You may find [chapter 8](https://web.stanford.edu/~jurafsky/slp3/8.pdf) of Jurafsky and Martin book useful. In particular, you could consider section 8.5.2 for features in NER.

<a name="features"></a>
## **Feature Engineering**
[[^^^]](#outline)
---

In [15]:
#import nltk
#nltk.download('averaged_perceptron_tagger')
#from nltk.tag import pos_tag
#sentence = ['The', 'cat', 'sat', 'on', 'the', 'wall', '.']
#print(pos_tag(sentence))

In [18]:
# Your implementation here
!pip install geotext
import nltk
nltk.download('averaged_perceptron_tagger_eng')
from geotext import GeoText
from nltk.tag import pos_tag

sentence = ['The', 'cat', 'sat', 'on', 'the', 'wall', '.']
print(pos_tag(sentence))

#Geotext Package
#NLTK.pos_tag
#Word
#Previous tag
#Has a number
#is first
#is last
#Capitalized


def feature_engineering(tokens_list, labels_list):
  unique_labels = ['O', 'B-LOC', 'B-PER', 'I-PER', 'B-ORG', 'B-MISC', 'I-LOC', 'I-MISC', 'I-ORG']
  #unique_labels = []
  #for label in labels_list:
  #  if label not in unique_labels:
  #   unique_labels.append(label)
  parts_of_speech = pos_tag(tokens_list)

  all_feature_vectors = []
  for token_i in range(len(tokens_list)):
    current_token = tokens_list[token_i]
    current_token_features = []
    #Feature 1: Geotext Package
    geoTrue = 0
    if len(GeoText(current_token).cities) != 0:
      geoTrue = 1
    if len(GeoText(current_token).countries) != 0:
      geoTrue = 1
    current_token_features.append(geoTrue)

    #Feature 2: Previous Label
    if token_i > 0:
      previous_label = labels_list[token_i - 1]
      previous_label_index = unique_labels.index(previous_label)
    else:
      previous_label_index = -1
    current_token_features.append(previous_label_index)

    #Feature 3: Has a number
    has_number = 0
    for letter in current_token:
      if letter.isdigit():
        has_number = 1
    current_token_features.append(has_number)

    #Feature 4: isCapitalized
    if current_token[0].isupper():
      current_token_features.append(1)
    else:
      current_token_features.append(0)

    #Feature 5: word
    current_token_features.append(tokens_list[token_i])

    #Feature 6: Previous Word Part of Speech
    if token_i > 0:
      previous_pos = parts_of_speech[token_i-1][1]
    else:
      previous_pos = -1
    current_token_features.append(previous_pos)

    #Feature 7: Current Word Part of Speech
    current_pos = parts_of_speech[token_i][1]
    current_token_features.append(current_pos)

    all_feature_vectors.append(current_token_features)
  return all_feature_vectors

#Turns the feature_vector of 1 sentence into dictionary format
#feature_vector is a list of the list of features for each token in a sentence
def changeFeatureVectorToDictionary(feature_vector):
  returnDictionaries = []
  for index in range(len(feature_vector)):
    current_token_features = feature_vector[index]
    addDict = {'GeoText' : current_token_features[0],\
               'PreviousLabel' : current_token_features[1],\
               'HasNumber' : current_token_features[2],\
               'isCapitalized' : current_token_features[3],\
               'word' : current_token_features[4]}
    returnDictionaries.append(addDict)
  return returnDictionaries

def createTrainingMaxEnt():
  labels = train.get('NER')
  text = train['text']
  print(len(text))

  print(labels)
  print(text)

  labels_in_one_list = []
  for sentence_i in range(len(labels)):
    current_sentence = labels[sentence_i]
    for label_i in range(len(current_sentence)):
      labels_in_one_list.append(current_sentence[label_i])

  list_of_dictionaries = []

  for sentence_i in range(len(text)):
    current_sentence_features = feature_engineering(text[sentence_i], labels[sentence_i])
    current_sentence_dictionary = changeFeatureVectorToDictionary(current_sentence_features)
  list_of_dictionaries += current_sentence_dictionary

  #make into dictionary label pairs
  alltuples = []
  for index in range(len(list_of_dictionaries)):
    paired_tuple = (list_of_dictionaries[index], labels_in_one_list[index])
    alltuples.append(paired_tuple)

  return alltuples

from nltk.classify import MaxentClassifier
train_input = createTrainingMaxEnt()
print(train_input)
model = MaxentClassifier.train(train_input, max_iter = 20)














[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


[('The', 'DT'), ('cat', 'NN'), ('sat', 'VBD'), ('on', 'IN'), ('the', 'DT'), ('wall', 'NN'), ('.', '.')]
7000
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', '

In [19]:
len(train['text'])
len(train.get('text'))

7000

<a name="memm_implementation"></a>
## **MEMM Implementation**
[[^^^]](#outline)
---

In [20]:
# Your implementation here
import numpy as np
from nltk.tag import pos_tag
def viterbi_memm(model, observation):

  #Get a list of all potential labels (Named Entity Mappings)
  labels_list = ['O', 'B-LOC', 'B-PER', 'I-PER', 'B-ORG', 'B-MISC', 'I-LOC', 'I-MISC', 'I-ORG']

  parts_of_speech = pos_tag(observation)
  #INITIAL DEBUGGING: print(labels_list)

  #Initialization
  #label_tracker: tracks the previous label that got to this token.
  #Saves the previous label as a number representing that label's index in labels_list
  #This will be a list within a list
  label_tracker = []
  #probabilities_tracker: tracks the probability to get to current token for every label
  #list of lists, contains a list for every token (size of inner list is the total
  #number of labels and contains the probability to get the current label for
  #the current token).
  probabilities_tracker = []

  #Initialize the start token
  #Initialize label_tracker for start token
  initial_label_tracker = [None] * len(labels_list)
  label_tracker.append(initial_label_tracker)
  #initialize probabilities_tracker for start token
  first_token = observation[0]
  first_token_probabilities = [] #probability first token has each label
  for label in range(len(labels_list)):
    current_label_probability = 0

    #Make feature vector
    first_token_features = []
    #Feature 1: Geotext Package
    geoTrue = 0
    if len(GeoText(first_token).cities) != 0:
      geoTrue = 1
    if len(GeoText(first_token).countries) != 0:
      geoTrue = 1
    first_token_features.append(geoTrue)

    #Feature 2: Previous Label
    first_token_features.append(-1)

    #Feature 3: Has a number
    has_number = 0
    for letter in first_token:
      if letter.isdigit():
        has_number = 1
    first_token_features.append(has_number)

    #Feature 4: isCapitalized
    if first_token[0].isupper():
      first_token_features.append(1)
    else:
      first_token_features.append(0)

    #Feature 5: word
    first_token_features.append(first_token)

    #Feature 6: Previous Word Part of Speech
    previous_pos = -1
    first_token_features.append(previous_pos)

    #Feature 7: Current Word Part of Speech
    current_pos = parts_of_speech[0][1]
    first_token_features.append(current_pos)

    feature_dictionary = changeFeatureVectorToDictionary([first_token_features])[0]
    probability_distribution_of_feature_vector = model.prob_classify(feature_dictionary)


    add_probability = probability_distribution_of_feature_vector.prob(labels_list[label])
    if add_probability <= 10**(-8):
      adjusted_probability = -100
    else:
      adjusted_probability = np.log(add_probability)
    previous_label_to_current_label_probability = adjusted_probability

    first_token_probabilities.append(current_label_probability)
  probabilities_tracker.append(first_token_probabilities)

  #loop through every token
  for i in range(1, len(observation)):
    current_token = observation[i]

    #Saved max probability of each current_label_probability.
    #So we have the max probability to get to current label for the current token
    current_token_probability = []
    #Saves the previous label to get to the current label for the current token
    current_token_previous_label = []

    #Recursion process to fill in the probabilities_tracker and label_tracker
    #loop through all the labels to update probabilities current token has
    #the current label
    for current_label in range(len(labels_list)):
      #current_label_probability is the probability to have the current label
      #for the current token given all previous tokens as potential paths
      current_label_probability = []
      #loop through all the previous labels to see the paths to get to
      #this current label
      for previous_label in range(len(labels_list)):

        #Make feature vector
        current_token_previous_label_features = []
        #Feature 1: Geotext Package
        geoTrue = 0
        if len(GeoText(current_token).cities) != 0:
          geoTrue = 1
        if len(GeoText(current_token).countries) != 0:
          geoTrue = 1
        current_token_previous_label_features.append(geoTrue)
        #Feature 2: Previous Label
        current_token_previous_label_features.append(previous_label)

        #Feature 3: Has a number
        has_number = 0
        for letter in current_token:
          if letter.isdigit():
            has_number = 1
        current_token_previous_label_features.append(has_number)

        #Feature 4: isCapitalized
        if current_token[0].isupper():
          current_token_previous_label_features.append(1)
        else:
          current_token_previous_label_features.append(0)

        #Feature 5: word
        current_token_previous_label_features.append(current_token)

        #Feature 6: Previous Word Part of Speech
        previous_pos = parts_of_speech[i-1][1]
        current_token_previous_label_features.append(previous_pos)

        #Feature 7: Current Word Part of Speech
        current_pos = parts_of_speech[i][1]
        current_token_previous_label_features.append(current_pos)

        feature_dictionary = changeFeatureVectorToDictionary([current_token_previous_label_features])[0]

        probability_distribution_of_feature_vector = model.prob_classify(feature_dictionary)

        add_probability = probability_distribution_of_feature_vector.prob(labels_list[current_label])
        if add_probability <= 10**(-8):
          adjusted_probability = -100
        else:
          adjusted_probability = np.log(add_probability)
        previous_label_to_current_label_probability = adjusted_probability

        current_label_probability.append(previous_label_to_current_label_probability)
      #Get the max_probability_path to get to current label for current token
      max_probability_path = get_max_of_list(current_label_probability)
      #Save this max_probability_path to the current_token_probability and
      #the current_token_previous_label
      current_token_probability.append(max_probability_path[0])
      current_token_previous_label.append(max_probability_path[1])

    #Save the current_token_probability and current_token_previous_paths into
    #our overall probabilities_tracker and label_tracker
    #INITIAL DEBUGGING: print(current_token_probability)
    #INITIAL DEBUGGING: print(current_token_previous_label)
    probabilities_tracker.append(current_token_probability)
    label_tracker.append(current_token_previous_label)

  #Termination process to get the path
  last_token_index = len(observation)-1
  #Gets the highest probability path to the last token
  max_probability_last_token = get_max_of_list(probabilities_tracker[last_token_index])
  #Index of Label for the last token with the highest probability path
  last_label_index = max_probability_last_token[1]
  last_label = labels_list[last_label_index]

  #Get all previous labels given the last label.
  #Loop through all tokens starting with the last token and then we prepend
  #to list using python insert at index 0.
  current_label_index = last_label_index
  optimal_path = [last_label]
  for i in range(len(observation)-1):
    current_token = len(observation) - i  - 1
    previous_label_index = label_tracker[current_token][current_label_index]
    optimal_path.insert(0, labels_list[previous_label_index])
    current_label_index = previous_label_index

  return optimal_path



#returns a list where the first element is the value of the highest value in
#the list and the second value is the index of the highest value.
def get_max_of_list(list):
  current_index = 0
  current_max = list[0]
  for i in range(len(list)):
    if list[i] > current_max:
      current_max = list[i]
      current_index = i
  to_return = [current_max, current_index]
  return to_return






### **Validation**
---
In this section we want you to run your MaxEnt model on the validation dataset you extracted earlier. We want you to play around with different combinations of features in order to find which features work the best for your implementation. You will be asked to write about this process in detail in written question 3.3 so please spend time experimenting with features! Once again, please use the code we provided for computing Entity Level Avg F1 earlier when validating your model.

In [21]:
# Run your model on validation set
# You will need to
# 1. Call your function above to get a prediction result on Validation Set
# 2. Report Metrics
# (See if you need to modify your feature set)

pred_token_labels = []
true_token_labels = []
token_indices = []

#Call viterbi_MEMM for every sentence
for sentence_i in range(len(val.get('text'))):
  sentence = val.get('text')[sentence_i]
  pred_token_labels += viterbi_memm(model, sentence)
  true_token_labels += val.get('NER')[sentence_i]
  token_indices += val.get('index')[sentence_i]

y_pred_dict = format_output_labels(pred_token_labels, token_indices)
#print("y_pred_dict is : " + str(y_pred_dict))
y_true_dict = format_output_labels(true_token_labels, token_indices)
#print("y_true_dict is : " + str(y_true_dict))

print("Entity Level Mean F1 score is : " + str(mean_f1(y_pred_dict, y_true_dict)))

Entity Level Mean F1 score is : 0.00213857998289136


<a name="q3.1"></a>
[[^^^]](#outline)

## **Q3.1: Implementation Details**
Explain how you implemented the MEMM and whether/how you modified Viterbi (e.g. which algorithms/data structures you used, what features are included). Make clear which parts were implemented from scratch vs. obtained via an existing package.


#### **A3.1:**

... add your answers here

I implemented my MEMM by defining multiple methods.
The method feature_engineering takes in a list of tokens and the corresponding list of labels and creates a feature vector for every token in that sentence. The feature vectors are created by the 7 features listed in the code. We then output a list of the feature vectors for every token in the sentence.

The method changeFeatureVectorToDictionary(feature_vector) takes in the feature vector of a sentence as a list and it returns the feature vector formatted as a tuple where the first entry of the tuple is a dictionary (key is the feature name and value is the value of the feature for the token) and the second entry of the tuple is the label for the word.

The createTrainingMaxEnt() method uses the training data to create a list of tuples where the first entry of the tuple is a dictionary (key is the feature name and value is the value of the feature for the token) and the second entry of the tuple is the label for the word. We call the feature_engineering method to create the feature vector and we call the changeFeatureVectorToDictionary(feature_vector) method to make the feature vector in the proper format.

Finally, we use the MaxentClassifer to turn our feature vectors from the training data into a proper maximum entropy model.

The next step is then to implement the viterbi algroithm. The implementation is very similar as the implementation for HMM. However, the difference is that rather than assigning probabilities with transitions and emissions matrix, we assign probabilities with the maximum entropy model. We do this by creating the feature vectors for every possible way that a previous label can get to the current label for the current token and we keep the max probability.

Packages that were used include the MaxentClassifer to train the data and the nltk.tag to get the parts of speech tag for each sentence.

<a name="q3.2"></a>
[[^^^]](#outline)

## **Q3.2: Results Analysis**
Explain here how you evaluated the MEMM model. Summarize the performance of your system and any variations that you experimented with the validation datasets. Put the results into clearly labeled tables or diagrams and include your observations and analysis.


#### **A3.2:**

... add your answers here

We evaluated the MEMM model using the validation data and seeing the error created with the specific features. Some variations we experimented with include having the word as a feature as well as part of speech tagging. We were able to increase our results because of this which was super helpful by adding more data. As a result, we were able to have a better kaggle performance by playing around with the feature vectors and removing any feature vector that wasn't necessary or adding more feature vectors until we had a better result with the validation data.

<a name="q3.3"></a>
[[^^^]](#outline)

## **Q3.3: Feature Engineering**
What features are considered most important by your MaxEnt Classifier? Why do you think these features make sense? Describe your experiments with feature sets. An analysis on feature selection for the MEMM is required – e.g. what features **help most**, why? An **error analysis** is required – e.g. what sorts of errors occurred, why?

#### **A3.3:**

... add your answers here

Some of the features considered most important by my MaxEnt Classifer is the word itself, capitalization, as well as the part of speech tagging. I think that the part of speech of a word can greatly change if that word is one of the names we want to label or if it isn't one of the names. Example, a lot of our names are probably nouns. Further, capitalization of the word can indicate if it is an important word taht needs to be named. Finally, the word itself can have an impact since some words are proper nouns with a specific defined definition.

Some errors that occured was in part of speech tagging where I realized that the data structure it took was a list of tuples for the entire sentence rather than the part of speech for the specific token. Thus, I needed to play around with the data structure to prevent the error from occuring and extract the necessary value.

Initially, I also had a feature of first word and last word of a sentence. However, I realized after testing that this data was drastically reducing the performance. As a result, I had to remove this feature. I believe the reason is that being the first or last word of a sentence might have some changes in the training data, but on the testing data, it might result in different tags due to a difference in context and thus result in a lower result.


<a name="q3.4"></a>
[[^^^]](#outline)

## **Q3.4: Room for Improvement**
When did the system work well, when did it fail and any ideas as to why? How might you improve the system?


#### **A3.4:**

... add your answers here

The system worked better as more variables were added. I believe that this is because the maximum entropy classifer has more data to create a better model and analyze the results on the test data. For example, when I added part of speech tagging, my results suddenly had a big improvement from the previous features that I was using.

Some times when the model failed was when I used features like beginning of sentence and end of sentence. These features were not as useful and had differences betweeen training and testing data context, and thus, it caused errors in my data. As a result, I eliminated this feature since it was not an accurate model.

I will improve the system by adding additional features. If I had more time to experiment, I'd consider adding features that were dependent on multiple factors. For example, a binary feature indicating if the previous word is a specific tag as well as if the current word is a specific part of speech. This is just one example of implementing multiple elements logically into a binary feature.

<a name="part4"></a>
# **Part 4: Comparing HMMs and MEMMs**
[[^^^]](#outline)

---

In this section you will be asked to analyze and compare the models you have developed!




<a name="q4.1"></a>
[[^^^]](#outline)

## **Q4.1: Result Comparison**
Compare here your results (validation scores) for your HMM and the MEMM. Which of them performs better? Why?

#### **A4.1:**

... add your answers here

For some reason, my HMM model performed better than my MEMM model, but this is probably because my use of features in the MEMM model was not as efficient as it could be. I believe if I added more features to my MEMM model, I can have the ability to do even better with my MEMM model than my HMM model.

For example, my kaggle validation score was over 0.5 for HMM while my MEMM kaggle validation score was just under 0.5.

However, at its present situation, my HMM model is currently the better model than MEMM. I believe this is because HMM was more straight forward to implement and thus had more consistent results. My MEMM model is very highly dependent on the features I chose, and thus, since I may not have chosen the best features or enough features, my HMM model did better.

I think MEMM has more potential since you can have a much more abundant amount of feature vectors. Also, you can overfit the data on the training set using descriptive modeling to have a better and more accurate fit for the current situation.

<a name="q4.2"></a>
[[^^^]](#outline)

## **Q4.2: Error Analysis 1**
Do some error analysis. What are error patterns you observed that the HMM makes but the MEMM does not? Try to justify why/why not? **Please give examples from the dataset.**

#### **A4.2:**

... add your answers here

The HMM model can only assess based on the previous NER and thus, if there is less correlation between previous and current tags, the HMM model has its limitations. The HMM model is only limited to information on the previous tag as well, so it can cause the data to not incorporate as much as it could. This could be beneficial for shorter pieces of text where long distance features are not needed. Some other errors that HMM makes is that it tends to predict a little too many 'O' labels and the reason for this is that in the transitions matrix, since there's a lot of 'O' words, the frequency for the next word to be an 'O' could possibly be greater.

<a name="q4.3"></a>
[[^^^]](#outline)

## **Q4.3: Error Analysis 2**
What are error patterns you observed that MEMM makes but the HMM does not? Try to justify what you observe? **Please give examples from the dataset.**

#### **A4.3:**

... add your answers here

Some error patterns I observed in the MEMM was in the choice of my feature vectors. Since the MEMM model requires you to define specific features for each token, it is a lot more open to interpretation, and thus, errors can arise if you pick the wrong feature vector. While this allows us to overfit the data to create a more precise model so that MEMM could be a better result than HMM since the feature structure is based on descriptive information that I as the coder implemented, it is also possible that we overfit the features to the training data in MEMM and it can be less applicable to test data. Overall, MEMM had a lot more errors in the choice of feature vectors since it is a lot more open to interpretation.

<a name="part5"></a>
# **Part 5: Kaggle Submission**
[[^^^]](#outline)

---

Using the best-performing system from among all of your HMM and MEMM models, generate predictions for the test set, and submit them to [Kaggle competition](https://www.kaggle.com/t/200697e4726f448986930dd4e823e957). Below, we provide a function that submits given predicted tokens and associated token indices in the correct format. As a scoring metric on Kaggle, we use **Entity Level Mean F1**.

Your submission to Kaggle should be a CSV file consisting of five lines and two columns. The first line is a fixed header, and each of the remaining four lines corresponds to one of the four types of named entities. The first column is the label identifier *Id* (one of PER, LOC, ORG or MISC), and the second column *Predicted* is a list of entities (separated by single space) that you predict to be of that type. Each entity is specified by its starting and ending index (concatenated by a hypen) as given in the test corpus.

You can use the function **create_submission** that takes the list of predicted labels and the list of associated token indices as inputs and creates the the output CSV file at a specified path.

NOTE: Ensure that there are **no** rows with *Id* = "O" in your Kaggle Submission.

In [22]:
import csv

def create_submission(output_filepath, token_labels, token_inds):
    """
    :parameter output_filepath: The full path (including file name) of the output file,
                                with extension .csv
    :type output_filepath: [String]
    :parameter token_labels: A list of token labels (eg. PER, LOC, ORG or MISC).
    :type token_labels: List[String]
    :parameter token_indices: A list of token indices (taken from the dataset)
                              corresponding to the labels in [token_labels].
    :type token_indices: List[int]
    """
    label_dict = format_output_labels(token_labels, token_inds)
    with open(output_filepath, mode='w') as csv_file:
        fieldnames = ['Id', 'Predicted']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for key in label_dict:
            p_string = " ".join([str(start)+"-"+str(end) for start,end in label_dict[key]])
            writer.writerow({'Id': key, 'Predicted': p_string})

path = "/content/drive/MyDrive/nlpthingy.csv"
###INITIAL DEBUGGING: print(test.get('text'))
###INITIAL DEBUGGING: print(test.get('index'))
hmm = build_hmm(train.get('text'), train.get('NER'))
###Currently, test.get('text') is a list of sentences. Convert to a list of words
text = test.get('text')
text_list = []
for sentence_i in range(len(test.get('text'))):
  sentence = text[sentence_i]
  for token in sentence:
    text_list.append(token)
#Currently, test.get('index') is a list of the list of indexes for a sentence.
#Convert to a list of indexes
index = test.get('index')
index_list = []
for sentence_i in range(len(test.get('index'))):
  sentence = index[sentence_i]
  for token in sentence:
    index_list.append(token)
output_labels = viterbi(hmm, text_list)

#FOR HMM
create_submission(path, output_labels, index_list)

#Call viterbi_MEMM for every sentence
output_labels_MEMM = []
for sentence_i in range(len(test.get('text'))):
  sentence = test.get('text')[sentence_i]
  output_labels_MEMM += viterbi_memm(model, sentence)
#create_submission(path, output_labels_MEMM, index_list)

## **Baselines**

On Kaggle, we provide two baselines for you to evaluate your models agaist: **`HMM Baseline`** and **`MEMM Baseline`**. You may use them to internally check your models. In addition, you may get points if for the final submission your best-performing model does better than the **`MEMM baseline`**.



---
<a name="q5"></a>
## **Q5: Competition Score**
[[^^^]](#outline)


Include your team's **best score** and the **name under which that best score was submitted** from Kaggle.

#### **A5:**

... add your answers here

0.54061 is best score, Name: Super Smash Bros